In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import matplotlib.pyplot as plt
import sys
import os
# import warnings
# warnings.filterwarnings('ignore')

#user = 'madams01'
#sys.path.append(f'/home/{user}/git_clones/copymltox/ml-tox/src')
sys.path.append("/home/ishah/ipynb/ml-tox/src")
from mltox.db.mongo import *
from mltox.db.bc import *
from mltox.db.bio import *
#from mltox.db.graph_utils import *

import functools
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# os.environ['CUDA_VISIBLE_DEVICES']=''
!CUDA_VISIBLE_DEVICES='0'
!export CUDA_VISIBLE_DEVICES
os.getenv('CUDA_VISIBLE_DEVICES')


# Data

## ATG

In [5]:
DAT='/home/ishah/ipynb/ml-tox/data/bio/'
CH_ATG = pd.read_csv(DAT+'chm-atg.csv')
CH_ATG

,dsstox_sid,smiles,ATG_AP_1_CIS,ATG_AP_2_CIS,ATG_AR_TRANS,ATG_Ahr_CIS,ATG_BRE_CIS,ATG_CAR_TRANS,ATG_CMV_CIS,ATG_CRE_CIS,...,ATG_Sp1_CIS,ATG_TAL_CIS,ATG_TA_CIS,ATG_TCF_b_cat_CIS,ATG_TGFb_CIS,ATG_THRa1_TRANS,ATG_VDRE_CIS,ATG_VDR_TRANS,ATG_Xbp1_CIS,ATG_p53_CIS
0,DTXSID0020232,CN1C=NC2=C1C(=O)N(C)C(=O)N2C,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,DTXSID0020236,[Ca++].CC(O)C([O-])=O.CC(O)C([O-])=O,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,DTXSID0020280,[O-][N+](=O)C1=C(Cl)C=CC=C1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,DTXSID0020282,NC1=CC(N)=C(Cl)C=C1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,DTXSID0020284,OS(O)(=O)=O.NC1=CC=C(N)C(Cl)=C1,1,0,1,1,0,0,1,1,...,1,1,1,0,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,DTXSID9051907,COC1=NC=CN=C1CC(C)C,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3475,DTXSID9052505,CCCCC(C)=C,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3476,DTXSID9052509,OC1=CC=C(C=C1)C#N,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3477,DTXSID9052846,COCC(C)N(C(=O)CS(O)(=O)=O)C1=CSC=C1C,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Features

In [6]:
import deepchem as dc
from deepchem.models.torch_models import AttentiveFPModel
#import scipy


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
/opt/miniconda3/envs/dc310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-05 17:27:33.974062: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 17:27:33.974115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 17:27:33.975392: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 17:2

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [7]:
import deepchem as dc
from deepchem.models import AttentiveFPModel
import scipy

Y_names = CH_ATG.columns[2:]
DS = dc.data.NumpyDataset(CH_ATG.smiles,CH_ATG.iloc[:,2:])
FT = dc.trans.FeaturizationTransformer(DS, dc.feat.MolGraphConvFeaturizer(use_edges=True))
DS = FT.transform(DS)

Failed to featurize datapoint 244, [Br-].[Br-].[Ca+2]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 272, [Cl-].[Cl-].[Cl-].[Yb+3]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 913, [Cl-].[Cl-].[Zn+2]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 932, [I-].[Na+]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 1505, [Cl-].[Cl-].[Pd+2]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 1769, [Cl-].[Cl-].[Hg+2]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 1979, [Cu+].[I-]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 2669, [Cl-].[Cl-].[Ni+2]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 3040, [C

In [7]:
CH_ATG

,dsstox_sid,smiles,ATG_AP_1_CIS,ATG_AP_2_CIS,ATG_AR_TRANS,ATG_Ahr_CIS,ATG_BRE_CIS,ATG_CAR_TRANS,ATG_CMV_CIS,ATG_CRE_CIS,...,ATG_Sp1_CIS,ATG_TAL_CIS,ATG_TA_CIS,ATG_TCF_b_cat_CIS,ATG_TGFb_CIS,ATG_THRa1_TRANS,ATG_VDRE_CIS,ATG_VDR_TRANS,ATG_Xbp1_CIS,ATG_p53_CIS
0,DTXSID0020232,CN1C=NC2=C1C(=O)N(C)C(=O)N2C,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,DTXSID0020236,[Ca++].CC(O)C([O-])=O.CC(O)C([O-])=O,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,DTXSID0020280,[O-][N+](=O)C1=C(Cl)C=CC=C1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,DTXSID0020282,NC1=CC(N)=C(Cl)C=C1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,DTXSID0020284,OS(O)(=O)=O.NC1=CC=C(N)C(Cl)=C1,1,0,1,1,0,0,1,1,...,1,1,1,0,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,DTXSID9051907,COC1=NC=CN=C1CC(C)C,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3475,DTXSID9052505,CCCCC(C)=C,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3476,DTXSID9052509,OC1=CC=C(C=C1)C#N,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3477,DTXSID9052846,COCC(C)N(C(=O)CS(O)(=O)=O)C1=CSC=C1C,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#[i for i in DS.X if i.num]
#[type(i) for i in list(DS.X)]
Keep=[]
for i,v in enumerate(DS.X):
    #print(i.num_nodes)
    try:
        n = v.num_nodes
    except:
        print(i,CH_ATG.dsstox_sid[i])
    else:
        Keep.append(i)

244 DTXSID0044577
272 DTXSID0047117
913 DTXSID2035013
932 DTXSID2041125
1505 DTXSID4025824
1769 DTXSID5020811
1979 DTXSID5041803
2669 DTXSID7040316
3040 DTXSID8041909
3341 DTXSID9036933


In [9]:
DS  = DS.select(Keep)

In [49]:
import glob, os
import tensorflow as tf 
import random 

s0 = 2314
tf.random.set_seed(s0)
random.seed(s0)
np.random.seed(s0)

Splitter = dc.splits.RandomStratifiedSplitter()
Perf = []
n_tasks = DS.y.shape[1]

for k_i,(DS_train,DS_test) in enumerate(Splitter.k_fold_split(DS,k=5)):
    print(k_i,DS_train.X.shape,DS_test.X.shape)
    AFP_k = AttentiveFPModel(mode='classification', 
                            n_tasks=DS_train.y.shape[1],
                            num_layers=2,
                            graph_feat_size=100,
                            batch_size=16, 
                            learning_rate=0.001)
    AFP_k.fit(DS_train,nb_epoch=40)
    Y_true = DS_test.y
    Y_pred = AFP_k.predict(DS_test)
    
    # ROC_k = pd.DataFrame([[Y_names[i],dc.metrics.roc_auc_score(dc.metrics.to_one_hot(Y_true[:,i],Y_pred[:,i]))]
    #                       for i in range(n_tasks)])
    ROC_k = pd.DataFrame([[Y_names[i],dc.metrics.roc_auc_score(Y_true[:,i],Y_pred[:,i,1])]
                      for i in range(n_tasks)],columns=['cls','auc'])

    Perf.append(ROC_k)
PERF = pd.concat(Perf)
P1 = PERF.groupby('cls').aggregate(dict(auc=[np.mean,np.std]))
P1.columns=[i[0]+'_'+i[1] for i in P1.columns]


0 (2775,) (694,)
1 (2775,) (694,)
2 (2775,) (694,)
3 (2775,) (694,)
4 (2776,) (693,)


/tmp/ipykernel_421315/234746466.py:33: FutureWarning: The provided callable <function mean at 0x7f75885acf70> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  P1 = PERF.groupby('cls').aggregate(dict(auc=[np.mean,np.std]))
/tmp/ipykernel_421315/234746466.py:33: FutureWarning: The provided callable <function std at 0x7f75885ad090> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  P1 = PERF.groupby('cls').aggregate(dict(auc=[np.mean,np.std]))


In [50]:
P1.sort_values('auc_mean',ascending=False)

,auc_mean,auc_std
cls,,
ATG_PPARg_TRANS,0.795634,0.017413
ATG_PXR_TRANS,0.789873,0.003116
ATG_VDRE_CIS,0.787409,0.018259
ATG_MRE_CIS,0.780961,0.013726
ATG_PBREM_CIS,0.779136,0.017541
...,...,...
ATG_M_19_CIS,0.592973,0.033115
ATG_M_32_TRANS,0.581052,0.018528
ATG_M_06_TRANS,0.561669,0.028369


# Train model on entire data set

In [ ]:
AFP_full = AttentiveFPModel(mode='classification', 
                            model_dir='/home/ishah/ipynb/ml-tox/models/atg-afp-1',
                            n_tasks=DS.y.shape[1],
                            num_layers=2,
                            graph_feat_size=100,
                            batch_size=16, 
                            learning_rate=0.001)
AFP_full.fit(DS,nb_epoch=40)
#AFP_full.save()

AttributeError: Can't pickle local object 'SparseSoftmaxCrossEntropy._create_pytorch_loss.<locals>.loss'